In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import xgboost as xgboost

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Approach 1: TimeSeries approach groupby sales date -> merge with google data
(Assumption: Avocado King wants to see sale price per day )

## Load sales dataset

In [ ]:
# Load dataset, groupby date and keep date, price, volume sales only
price_df = pd.read_csv('Data/price-and-sales-data.csv')

price_df = price_df[['Date', 'TotalVolume', '4046', '4225', '4770', 'AveragePrice']]
price_df.head()

In [ ]:
price_df.info()

In [ ]:
price_df.isna().sum()

## Handle missing values with median
(one practice since only numeric columns selected)

In [ ]:
price_df.fillna(price_df.median(numeric_only=True), inplace=True)

In [ ]:
price_df.isna().sum()

In [ ]:
price_df['Date'] = pd.to_datetime(price_df['Date'])

## Group by date

In [ ]:
weekly_sales = price_df.groupby('Date').mean()

In [ ]:
weekly_sales.head(20)

In [ ]:
weekly_sales['AveragePrice'].plot()

In [ ]:
google_data = pd.read_csv('Data/google-data.csv')
google_data['Week'] = pd.to_datetime(google_data['Week'])
google_data.head(20)

## Merge with Google data

In [ ]:
combined_df = pd.merge(weekly_sales, google_data, how='outer', left_on='Date', right_on='Week')
combined_df.head(20)

## Reason missing values with merged data

In [ ]:
combined_df.isnull().sum()

In [ ]:
combined_df[combined_df.isnull().any(axis=1)]

In [ ]:
combined_df.dropna(inplace=True)

## Feature selection based on correlation

In [ ]:
import numpy as np
sns.heatmap(np.abs(combined_df.corr()),annot=True, cmap='coolwarm')

In [ ]:
a = np.abs(combined_df.corr())['AveragePrice'] > 0.3
column_df = pd.DataFrame(a)
keep_cols = list(column_df[column_df.AveragePrice==True].index.values)


In [ ]:
keep_cols

In [ ]:
combined_df

## Model training

### Train test split

In [ ]:
def train_test_split_timeseries(input_df, test_start=2019, index_col='Week', output_col='AveragePrice', keep_cols=None):    
    set_df = input_df.set_index(index_col)
    if keep_cols:
        set_df = set_df[keep_cols]

    X = set_df.drop([output_col], axis=1)
    y = set_df[output_col]

    X_train = X[:str(test_start-1)]
    y_train = y[:str(test_start-1)]
    y_test = y[str(test_start):]
    X_test = X[str(test_start):]    
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split_timeseries(combined_df, test_start=2017, keep_cols=None)

### Scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_Xtrain = scaler.fit_transform(X_train)
scaled_Xtest = scaler.transform(X_test)

### Linear Regression model

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(scaled_Xtrain, y_train)
y_predicted = lr.predict(scaled_Xtest)
print('r2 score', r2_score(y_predicted, y_test.values))
result_plot(y_test.values, y_predicted, 'Timeseries regression')
sns.displot(np.abs((y_predicted-y_test.values)), label='Residual plot')
y_predicted_df = pd.DataFrame(y_predicted)
y_predicted_df.index = y_test.index
plt.figure(figsize=(20,8))
plt.plot(y_test)
plt.plot(y_predicted_df)

### Ridge modelling

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
alphas = [1e-3, 1e-2, .1, 1, 10, 1e2]
plt.figure(figsize=(16,8))
plt.plot(y_test)
labels = ['True value']
for (ii, alpha) in enumerate(alphas):
    model = Ridge(alpha=alpha)
    model.fit(scaled_Xtrain, y_train)
    y_predicted = model.predict(scaled_Xtest)
    y_predicted_df = pd.DataFrame(y_predicted)
    y_predicted_df.index = y_test.index
    print(alpha, r2_score(y_test.values, y_predicted), mean_squared_error(y_test.values, y_predicted))
    plt.plot(y_predicted_df)
    labels.append('Model with alpha='+str(alpha))

plt.legend(labels)

### Conclusion
Good for recognising the trend but not really predict the exact price. Be careful to input great incidence into the model

# Approach 2: Tabular approach

In [ ]:
price_df = pd.read_csv('Data/price-and-sales-data.csv')
price_df.head(100)

## Preprocessing

### Filling missing values with mean of that date

In [ ]:
# keep column names with missing values
checking_missing_values = price_df.isnull().any()
missing_value_cols = checking_missing_values[checking_missing_values == True].index.values
missing_value_cols

In [ ]:
for col in missing_value_cols:
    if price_df[col].dtypes != 'object': # double check dtype is NOT object (categorical)
        price_df[col] = price_df.groupby('Date')[col].transform(lambda x : x.fillna(x.mean()))
price_df.isnull().sum()

In [ ]:
print(price_df.describe())
print(price_df.info())

### Merge with Google data

In [ ]:
google_data = pd.read_csv('Data/google-data.csv')
google_data.head(10)

In [ ]:
combined_df = pd.merge(price_df, google_data, how='inner', left_on='Date', right_on='Week')
combined_df.head(100)

In [ ]:
combined_df.isnull().sum()

### Correlation analysis of combined data

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(combined_df.corr(), cmap='coolwarm', annot=True)

### One hot encoding

In [ ]:
cols = ['type', 'region']
combined_df[cols] = combined_df[cols].astype('category')
onehot_df = pd.get_dummies(combined_df, drop_first=True, columns=cols)
onehot_df.columns

In [ ]:
onehot_df.head(20)

### PCA

In [ ]:
X = onehot_df.drop(['Date', 'AveragePrice', 'Week'], axis=1)
pca = PCA(n_components=10)
pca.fit_transform(X)

var = pca.explained_variance_ratio_
cumsum_var = np.cumsum(var)
plt.plot(cumsum_var)

### Train test split

In [ ]:
def train_test_avocado(input_df, test_size=0.3, random_state=42, pca_component_keep=None, min_max_scaling=False):
    X = input_df.drop(['Date', 'AveragePrice', 'Week'], axis=1)
    y = input_df['AveragePrice']
    
    if pca_component_keep:
        pca = PCA(n_components = pca_component_keep)
        X = pca.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    if min_max_scaling:
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
    print('Training shape: ', X_train.shape)
    print('Testing shape: ', X_test.shape)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_avocado(onehot_df, test_size=0.3, random_state=42, pca_component_keep=None, min_max_scaling=True)

# Regression result plot

In [ ]:
import matplotlib
from sklearn.metrics import r2_score, \
        explained_variance_score, mean_absolute_error, median_absolute_error, mean_squared_log_error, mean_squared_error

def regression_results(y_true_a, y_pred_a):
    all_positive = ((y_true_a >= 0).all() and (y_pred_a >= 0).all())
    # Regression metrics
    l_explained_variance=explained_variance_score(y_true_a, y_pred_a)
    l_mean_absolute_error=mean_absolute_error(y_true_a, y_pred_a)
    l_mean_squared=mean_squared_error(y_true_a, y_pred_a)
    l_median_absolute_error=median_absolute_error(y_true_a, y_pred_a)
    l_r2=r2_score(y_true_a, y_pred_a)

    print('explained_variance: ', round(l_explained_variance,4))
    print('r2: ', round(l_r2,4))
    print('MAE: ', round(l_mean_absolute_error,4))
    print('MSE: ', round(l_mean_squared,4))
    print('RMSE: ', round(np.sqrt(l_mean_squared),4))
    print('median_absolute_error: ', round(l_median_absolute_error,4))
    if (all_positive):
        l_mean_squared_log_error=mean_squared_log_error(y_true_a, y_pred_a)
        print('mean_squared_log_error: ', round(l_mean_squared_log_error,4))
def result_plot(y_test_b, y_pred_b, name):
    plt.figure(figsize=(40,10))
    plt.plot(y_pred_b, 'ro', label='prediction')
    plt.plot(y_test_b,' go', label='ground truth')
    plt.legend(loc='upper right')
    plt.show()
    print(f'\n {name}:')
    regression_results(y_test_b, y_pred_b)
    
    matplotlib.rc('xtick', labelsize=15)
    matplotlib.rc('ytick', labelsize=15)

    fig, ax = plt.subplots(figsize=(10, 10))

    plt.style.use('ggplot')
    plt.plot(y_pred_b, y_test_b, 'ro')
    plt.xlabel('Predictions', fontsize = 15)
    plt.ylabel('Reality', fontsize = 15)
    plt.title('Predictions x Reality on dataset', fontsize = 15)
    ax.plot([y_pred_b.min(), y_pred_b.max()], [y_test_b.min(), y_test_b.max()], 'k--')
    plt.show()

# Modelling

### Decison Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dctr = DecisionTreeRegressor()
dctr.fit(X_train, y_train)
y_predicted = dctr.predict(X_test)
result_plot(y_test.values, y_predicted, 'Decision Tree')

### XGBoost Regressor

In [ ]:
xgbr = xgboost.XGBRegressor()
xgbr.fit(X_train, y_train)
y_predicted = xgbr.predict(X_test)
result_plot(y_test.values, y_predicted, 'XGBoost Regressor')

In [ ]:
# Plot feature importance but only PCA is not applied
# dic = {}
# for (col, score) in zip(X_train.columns, xgbr.feature_importances_):
#     dic[col] = [score]

# ft_imp = pd.DataFrame.from_dict(dic)
# ft_imp.iloc[0].plot(kind='bar', figsize=(16,8), title='Feature importance')

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_predicted = lr.predict(X_test)
result_plot(y_test.values, y_predicted, 'Linear Regression')

## Dense ANN modeling

In [ ]:
from tensorflow.keras import backend as K

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true-y_pred))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon())) 

def build_model(train_shape, optimizer='Adam', num_layers=1, num_perceptrons=10, activation='sigmoid'):
    model = Sequential()
    model.add(Input(shape=train_shape))
    for i in range(num_layers):
        model.add(Dense(num_perceptrons, activation=activation))
    model.add(Dense(1, activation='linear'))

    print(model.summary())
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse', 'mae', coeff_determination])
    return model

model = build_model(X_train.shape[1], num_layers=4, num_perceptrons=50)

history = model.fit(X_train, y_train, batch_size=32, epochs=50)

y_predicted = model.predict(X_train)
result_plot(y_test.values, y_predicted, 'Dense ANN')